In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# imports
import numpy as np 
import matplotlib.pyplot as plt
import pickle

In [2]:
import os
os.chdir(r'/home/ilaria/Scrivania/Machine_Learning/Project_2/Project2/data/twitter_datasets_epfl/unique/short')


# Doc2Vec by itself (build its vocab with Word2Vec)

Building the Doc2Vec model following the example:https://www.zybuluo.com/HaomingJiang/note/462804

In [10]:
from gensim.models.doc2vec import TaggedDocument,Doc2Vec

### Positive

In [14]:
fname = 'all_short.txt'

TotalNum_Unlabed = sum(1 for line in open(fname))
TestNum = 10000